In [ ]:
import torch
from src import criterion, dataset, metric, model, utils

In [ ]:
criterion_dice = criterion.DiceLoss()
criterion_bce = torch.nn.BCELoss()

In [ ]:
import pytorch_lightning as pl
class UNet(model.UNet, pl.LightningModule):
    def training_step(self, batch, batch_idx):
        images = batch['images']
        masks = batch['masks']
        # cracks = batch['cracks']
        outputs = self.forward(images)
        loss = criterion_dice(outputs, masks)
        # self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        images = batch['images']
        masks = batch['masks']
        # cracks = batch['cracks']
        outputs = self.forward(images)
        loss = criterion_dice(outputs, masks)
        # self.log('valid_loss', loss)
        return {'loss': loss}

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=1e-3)

In [ ]:
datasets = {
    'train': dataset.CracksDataset('train'),
    'valid': dataset.CracksDataset('valid'),
}
loaders = {
    'train': datasets['train'].get_loader(batch_size=2, shuffle=True, drop_last=True),
    'valid': datasets['valid'].get_loader(batch_size=2),
}

In [ ]:
net = UNet()
utils.get_gpu_memory_info()

In [ ]:
trainer = pl.Trainer(max_epochs=3)
trainer.fit(net, loaders['train'], loaders['valid'])